In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.profiler import profile, ProfilerActivity
from google.colab import drive
drive.mount('/content/drive')  # te pedirá permisos [web:50]

Mounted at /content/drive


In [ ]:
import os
digits = load_digits()
X,Y = digits.data,digits.target
X_train,X_test,Y_train,Y_test = train_test_split(X ,Y , test_size= 0.2, random_state=11)


train_data_set = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                               torch.tensor(Y_train))
test_data_set = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                               torch.tensor(Y_test))


train_data_loader = DataLoader(train_data_set,
                               batch_size=64,
                               shuffle=True)

test_data_loader = DataLoader(test_data_set,
                               batch_size=64,
                               shuffle=True)
model = nn.Sequential(nn.Linear(64,128),
                      nn.ReLU(),
                      nn.Linear(128,10))
optimizer=optim.SGD(model.parameters(), lr= 0.001, momentum = 0.9)
loss_fn= nn.CrossEntropyLoss()
it = iter(train_data_loader)
next(it)                      # warmup
features, targets = next(it)  # batch estable

activities = [ProfilerActivity.CPU]
with profile(activities=activities, with_stack=True, with_modules=True) as prof:
    optimizer.zero_grad()
    outputs = model(features)
    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()

# Ruta directa al Escritorio (home/Desktop) y creación de carpeta si existe
drive_dir = "/content/drive/MyDrive/Colab Notebooks"
os.makedirs(drive_dir, exist_ok=True)

trace_path = os.path.join(drive_dir, "trace.json")
prof.export_chrome_trace(trace_path)  # escribe el archivo en esa ruta [web:47]

